In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="ISO-8859-1", 
                 names=["target", "id", "date", "flag", "user", "text"])

# Keep only sentiment and text columns
df = df[["target", "text"]]

# Convert sentiment labels (0 = Negative, 4 = Positive) to (0, 1)
df["target"] = df["target"].map({0: 0, 4: 1})

# Reduce dataset size for faster testing (Optional)
df = df.sample(n=100000, random_state=42)  # Adjust size as needed

# Download stopwords
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

# Text preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # Remove URLs
    text = re.sub(r"@\w+|\#", "", text)  # Remove mentions and hashtags
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Remove special characters
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

df["clean_text"] = df["text"].apply(preprocess_text)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df["clean_text"], df["target"], test_size=0.1, random_state=42)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
vectorizer = TfidfVectorizer(max_features=5000)  # Use top 5000 features
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [3]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(max_iter=500)
log_reg.fit(X_train_tfidf, y_train)

y_pred = log_reg.predict(X_test_tfidf)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Logistic Regression Accuracy: 0.7627
              precision    recall  f1-score   support

           0       0.78      0.74      0.76      5045
           1       0.75      0.78      0.77      4955

    accuracy                           0.76     10000
   macro avg       0.76      0.76      0.76     10000
weighted avg       0.76      0.76      0.76     10000



In [6]:
from sklearn.svm import SVC

svm_model = SVC(kernel="linear")
svm_model.fit(X_train_tfidf, y_train)

y_pred = svm_model.predict(X_test_tfidf)
print("SVM Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


SVM Accuracy: 0.7611
              precision    recall  f1-score   support

           0       0.78      0.73      0.76      5045
           1       0.74      0.79      0.77      4955

    accuracy                           0.76     10000
   macro avg       0.76      0.76      0.76     10000
weighted avg       0.76      0.76      0.76     10000



In [5]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(X_train_tfidf, y_train)

y_pred = rf_model.predict(X_test_tfidf)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Random Forest Accuracy: 0.7457
              precision    recall  f1-score   support

           0       0.75      0.75      0.75      5045
           1       0.74      0.75      0.74      4955

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.75     10000
weighted avg       0.75      0.75      0.75     10000



In [4]:
from sklearn.naive_bayes import MultinomialNB

nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)

y_pred = nb_model.predict(X_test_tfidf)
print("Naïve Bayes Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Naïve Bayes Accuracy: 0.7484
              precision    recall  f1-score   support

           0       0.75      0.74      0.75      5045
           1       0.74      0.75      0.75      4955

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.75     10000
weighted avg       0.75      0.75      0.75     10000



In [2]:
import joblib

# Save the best model
joblib.dump(svm_model, "sentiment_model.pkl")

# Save the vectorizer
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")


NameError: name 'svm_model' is not defined

In [8]:
import joblib
# Load the model
model = joblib.load("sentiment_model.pkl")
vectorizer = joblib.load("tfidf_vectorizer.pkl")

# Predict on new tweets
new_tweet = ["i am depressed"]
new_tweet_tfidf = vectorizer.transform(new_tweet)

prediction = model.predict(new_tweet_tfidf)
print("Sentiment:", "Positive" if prediction[0] == 1 else "Negative")


Sentiment: Negative
